In [1]:
library(bigsnpr)
library(tidyverse)
library(stringr)

Loading required package: bigstatsr

── Attaching packages ─────────────────────────────────────── tidyverse 1.3.0 ──

✔ ggplot2 3.3.3     ✔ purrr   0.3.4
✔ tibble  3.0.6     ✔ dplyr   1.0.4
✔ tidyr   1.1.2     ✔ stringr 1.4.0
✔ readr   1.4.0     ✔ forcats 0.5.1

── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()



In [ ]:
gwas_raw <- read.csv("exBTs_regenie_phenoCol1_SPA_CADComp.regenie", TRUE, " ")

In [3]:
names(gwas_raw) <- c("chr", "pos", "rsid", "a0", "a1", "a1_freq", "info", "n_eff", "test", "beta", "beta_se", "chi_sq", "log_p")
map <- readRDS("/mnt/stsi/stsi1/ptseng/LD_ref_European_privefl/map.rds")
map <- map[(1:4)]
names(map) <- c('chr','pos','a0','a1')
#str(map)
#str(gwas_data)

In [16]:
#write.csv(gwas_raw,"regenie-ukbb_impute4_step2_f.50.0.0.regenie", row.names = FALSE)
#gwas_data <- gwas_raw[gwas_raw$LOG10P > 100,]
gwas_data <- gwas_raw
#gwas_data <- gwas_raw[gwas_raw$SNP %in% ldpred_data$sid & gwas_raw$,]
#gwas_data <- snp_match(gwas_raw, map)

In [17]:
gwas_data

chr,pos,rsid,a0,a1,a1_freq,info,n_eff,test,beta,beta_se,chi_sq,log_p
<int>,<int>,<chr>,<chr>,<chr>,<dbl>,<dbl>,<int>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>
1,10177,rs367896724,A,AC,0.40078300,0.467935,487275,ADD,-0.007136390,0.00968069,0.543431000,0.33628600
1,10352,rs201106462,T,TA,0.39461900,0.447900,487275,ADD,0.007334210,0.00994752,0.543596000,0.33635000
1,10616,1:10616_CCGCCGTTGCAAAGGCGCGCCG_C,CCGCCGTTGCAAAGGCGCGCCG,C,0.99416700,0.468135,487275,ADD,-0.060651900,0.06204040,0.955738000,0.48377800
1,11008,rs575272151,C,G,0.08659200,0.495022,487275,ADD,-0.011910500,0.01643370,0.525277000,0.32919800
1,11012,rs544419019,C,G,0.08659200,0.495022,487275,ADD,-0.011910500,0.01643370,0.525277000,0.32919800
1,13110,rs540538026,G,A,0.05723740,0.391826,487275,ADD,-0.002173510,0.02227290,0.009522960,0.03514600
1,13116,rs62635286,T,G,0.18512400,0.406244,487275,ADD,0.007242970,0.01316910,0.302499000,0.23483800
1,13118,rs62028691,A,G,0.18512400,0.406244,487275,ADD,0.007242970,0.01316910,0.302499000,0.23483800
1,13273,rs531730856,G,C,0.13214700,0.390799,487275,ADD,-0.013734800,0.01538110,0.797382000,0.42960100


In [18]:
don <- gwas_data %>% 
  
  # Compute chromosome size
  group_by(chr) %>% 
  summarise(chr_len=max(pos)) %>% 
  
  # Calculate cumulative position of each chromosome
  mutate(tot=cumsum(as.numeric(chr_len))-chr_len) %>%
  select(-chr_len) %>%
  
  # Add this info to the initial dataset
  left_join(gwas_data, ., by=c("chr"="chr")) %>%
  
  # Add a cumulative position of each SNP
  arrange(chr, pos) %>%
  mutate( BPcum=pos+tot)

axisdf = don %>% group_by(chr) %>% summarize(center=( max(BPcum) + min(BPcum) ) / 2 )

manplot <- ggplot(don, aes(x=BPcum, y=log_p)) +
    
    # Show all points
    geom_point( aes(color=as.factor(chr)), alpha=1, size=0.1) +
    scale_color_manual(values = rep(c("grey", "skyblue"), 22 )) +
    
    # custom X axis:
    scale_x_continuous( label = axisdf$chr, breaks= axisdf$center ) +
    scale_y_continuous(expand = c(0, 0) ) +     # remove space between plot area and x axis
  
    # Custom the theme:
    theme_bw() +
    theme( 
      legend.position="none",
      panel.border = element_blank(),
      panel.grid.major.x = element_blank(),
      panel.grid.minor.x = element_blank()
    )

In [19]:
#manplot

In [20]:
ggsave('regenie_manhattan_plot.png', plot = manplot, dpi=320, width = 10, height = 4, units = "in")

In [10]:
#write.csv(gwas_raw,"regenie-ukbb_impute4_step2_f.50.0.0.regenie", row.names = FALSE)
#gwas_data <- gwas_raw[gwas_raw$LOG10P > 100,]
#gwas_data <- gwas_raw
#gwas_data <- gwas_raw[gwas_raw$SNP %in% ldpred_data$sid & gwas_raw$,]
gwas_data <- snp_match(gwas_raw, map)

11,477,146 variants to be matched.

0 ambiguous SNPs have been removed.

1,054,139 variants have been matched; 0 were flipped and 0 were reversed.



In [11]:
gwas_data

chr,pos,a0,a1,rsid,a1_freq,info,n_eff,test,beta,beta_se,chi_sq,log_p,_NUM_ID_.ss,_NUM_ID_
<int>,<int>,<chr>,<chr>,<chr>,<dbl>,<dbl>,<int>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<int>,<int>
1,752721,A,G,rs3131972,0.8262400,1.000000,487275,ADD,-0.008524400,0.00896079,9.04972e-01,0.46667000,463,1
1,754182,A,G,rs3131969,0.8572630,0.990838,487275,ADD,-0.005653340,0.00956080,3.49641e-01,0.25624200,473,2
1,760912,C,T,rs1048488,0.8295880,0.990920,487275,ADD,-0.007790820,0.00892284,7.62360e-01,0.41726500,529,3
1,768448,G,A,rs12562034,0.1112660,1.000000,487275,ADD,0.000509008,0.01032120,2.43212e-03,0.01742710,557,4
1,779322,A,G,rs4040617,0.1365640,1.000000,487275,ADD,0.004130860,0.00957148,1.86262e-01,0.17649600,607,5
1,838555,C,A,rs4970383,0.2512950,1.000000,487275,ADD,0.002599250,0.00748859,1.20475e-01,0.13755800,838,6
1,846808,C,T,rs4475691,0.2021360,0.996894,487275,ADD,0.001033700,0.00810762,1.62555e-02,0.04645920,893,7
1,853954,C,A,rs1806509,0.6016330,0.997892,487275,ADD,-0.006024950,0.00665795,8.18892e-01,0.43710700,941,8
1,854250,A,G,rs7537756,0.2108940,0.998010,487275,ADD,0.002495620,0.00796231,9.82380e-02,0.12265500,942,9


In [12]:
don <- gwas_data %>% 
  
  # Compute chromosome size
  group_by(chr) %>% 
  summarise(chr_len=max(pos)) %>% 
  
  # Calculate cumulative position of each chromosome
  mutate(tot=cumsum(as.numeric(chr_len))-chr_len) %>%
  select(-chr_len) %>%
  
  # Add this info to the initial dataset
  left_join(gwas_data, ., by=c("chr"="chr")) %>%
  
  # Add a cumulative position of each SNP
  arrange(chr, pos) %>%
  mutate( BPcum=pos+tot)

axisdf = don %>% group_by(chr) %>% summarize(center=( max(BPcum) + min(BPcum) ) / 2 )

manplot <- ggplot(don, aes(x=BPcum, y=log_p)) +
    
    # Show all points
    geom_point( aes(color=as.factor(chr)), alpha=1, size=0.1) +
    scale_color_manual(values = rep(c("grey", "skyblue"), 22 )) +
    
    # custom X axis:
    scale_x_continuous( label = axisdf$chr, breaks= axisdf$center ) +
    scale_y_continuous(expand = c(0, 0) ) +     # remove space between plot area and x axis
  
    # Custom the theme:
    theme_bw() +
    theme( 
      legend.position="none",
      panel.border = element_blank(),
      panel.grid.major.x = element_blank(),
      panel.grid.minor.x = element_blank()
    )

In [13]:
#manplot

In [15]:
ggsave('regenie_manhattan_plot_reduced.png', plot = manplot, dpi=320, width = 10, height = 4, units = "in")

In [ ]:
gwas_raw